In [1]:
import torch
import torch.nn as nn

class TransformerDecoderLayer(nn.Module):
    def __init__(self, d_model, d_ff, num_heads):
        super().__init__()
        self.attention = nn.MultiheadAttention(d_model, num_heads, batch_first=True)
        self.xattention = nn.MultiheadAttention(d_model, num_heads, batch_first=True)
        self.ff_proj = nn.Linear(d_model, d_ff)
        self.output_proj = nn.Linear(d_ff, d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.act = nn.ReLU()

    def forward(self, x, y):
        """Process the input sequence x with decoder input y

        Args:
            x (torch.Tensor): The input sequence of shape (batch_size, seq_len, d_model).
            y (torch.Tensor): The output sequence from encoder of shape (batch_size, seq_len, d_model).

        Returns:
            torch.Tensor: The processed sequence of shape (batch_size, seq_len, d_model).
        """
        # Self-attention sublayer
        residual = x
        x = self.norm1(x)
        x = self.attention(x, x, x)
        x = x[0] + residual

        # Cross-attention sublayer
        residual = x
        x = self.norm2(x)
        x = self.xattention(x, y, y)
        x = x[0] + residual

        # Feed-forward sublayer
        residual = x
        x = self.norm3(x)
        x = self.act(self.ff_proj(x))
        x = self.act(self.output_proj(x))
        x = x + residual

        return x

dec_seq = torch.randn(3, 7, 16)
enc_seq = torch.randn(3, 11, 16)
layer = TransformerDecoderLayer(16, 32, 4)
out_seq = layer(dec_seq, enc_seq)
print({name: weight.shape for name, weight in layer.state_dict().items()})
print(out_seq.shape)

{'attention.in_proj_weight': torch.Size([48, 16]), 'attention.in_proj_bias': torch.Size([48]), 'attention.out_proj.weight': torch.Size([16, 16]), 'attention.out_proj.bias': torch.Size([16]), 'xattention.in_proj_weight': torch.Size([48, 16]), 'xattention.in_proj_bias': torch.Size([48]), 'xattention.out_proj.weight': torch.Size([16, 16]), 'xattention.out_proj.bias': torch.Size([16]), 'ff_proj.weight': torch.Size([32, 16]), 'ff_proj.bias': torch.Size([32]), 'output_proj.weight': torch.Size([16, 32]), 'output_proj.bias': torch.Size([16]), 'norm1.weight': torch.Size([16]), 'norm1.bias': torch.Size([16]), 'norm2.weight': torch.Size([16]), 'norm2.bias': torch.Size([16]), 'norm3.weight': torch.Size([16]), 'norm3.bias': torch.Size([16])}
torch.Size([3, 7, 16])
